In [3]:
import numpy as np
import pandas as pd

import re

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.pandas as ps

/usr/local/spark/python/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


## Extract 16GB worth of data

In [ ]:
filepath = '/mnt/data/public/opendota/matches.gz'
destination = joblib.load('csv_filepath.pkl')

In [ ]:
! zcat {filepath} | head -c 16G {destination}

## Create parquet files from 16GB data

In [2]:
spark = (
    SparkSession
    .builder
    .master('local[*]')
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/13 04:16:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_dota = spark.read.csv(
    'data/matches-16gb.csv',
    header=True
)

In [4]:
(
    df_dota
    .write
    .parquet(joblib.load('parquet_filepath.pkl'),
             mode='overwrite',
             partitionBy=['lobby_type'])
)

23/05/13 04:16:34 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## Read parquet

In [5]:
df_dota_parq = spark.read.parquet(
    joblib.load('parquet_filepath.pkl')
)

In [7]:
df_dota_parq.count()

1780371